In [1]:
from typing import Any
import pyodbc
from openpyxl import load_workbook
import csv
import re
import pdb
from datetime import datetime
import time


def get_data_from_RFB(consulta):
    # buscando dados na RFB
    query = "SELECT\
                RESPONSAVEL_CPF,\
                RESPONSAVEL_NOME,\
                END_LOGRADOURO, \
                END_NUMERO,\
                END_BAIRRO,\
                END_CEP,\
                END_MUNICIPIO,\
                END_UF,\
                DDD1,\
                TELEFONE1,\
                DDD2,\
                TELEFONE2,\
                EMAIL,\
                NOME_EMPRESARIAL,\
                NOME_FANTASIA, \
                DT_ABERTURA_ESTAB,\
                CNAE_PRINCIPAL_COD,\
                OPCAO_MEI,\
                PORTE,\
                SIT_CADASTRAL,\
                CNPJ \
            FROM dbo.VW010_EMPRESAS_RFB \
        WHERE RESPONSAVEL_CPF IN {0};".format(consulta)

    try:
        cursor_sql_server.execute(query)
        return cursor_sql_server.fetchall()
    except:
        #print('erro ',cpf)
        printConnectionError()
        connectBD()
        get_data_from_RFB(consulta)


def get_porte(optante_mei, porte_id):
    porte_dict = {
        0: 'Não informado',
        '00': 'Não informado',
        '0': 'Não informado',
        1: 'ME',
        '01': 'ME',
        '1': 'ME',
        3: 'EPP',
        '03': 'EPP',
        '3': 'EPP',
        5: 'Demais',
        '05': 'Demais',
        '5': 'Demais'
    }
    #print('porte_id = ',porte_id)
    if(optante_mei == 'S'):
        return 'MEI'
    elif porte_id != '' and porte_id is not None and porte_dict[porte_id] is not None:
        return porte_dict[porte_id]
    else:
        return ''


def get_fantasy_name(razao_social, nome_fantasia):
    if(nome_fantasia is None or nome_fantasia == ''):
        return razao_social
    return nome_fantasia


def connectBD():
    # configurando conexão SQL SERVER
    try:
        print('Iniciando conexão com banco de dados')
        global sql_server_conection
        global cursor_sql_server
        sql_server_conection=pyodbc.connect(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=10.1.140.172;DATABASE=DH1MVP_HUB;UID=qlik-sebrae-sc;PWD=$3bR@eBISC!2021;')
        cursor_sql_server=sql_server_conection.cursor()
        
        print('Conectado')
    except:
        printConnectionError()
        time.sleep(60)
        connectBD()


def closeConnectionDB():
    cursor_sql_server.close()
    sql_server_conection.close()


def printConnectionError():
    print('Erro Conexão: '+datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


sql_server_conection: Any
cursor_sql_server: Any
connectBD()

# lendo planilha
print('Iniciando leitura da planilha de cpfs')

wb=load_workbook(filename='cpfs.xlsx', read_only=True)  # planilha
ws=wb['script']  # Lendo aba da planilha

# criando csv
with open('cnpjs_enriquecidos_infinito.csv', 'w', newline='') as newFile:
    newFileWriter=csv.writer(newFile)
    cpfs=[]

    newFileWriter.writerow([
      'CPF', #cpf
      'CONTATO', #contato
      'ENDERECO', # endereco
      'NUMERO', # numero
      'BAIRRO', # bairro
      'CEP',
      'MUNICIPIO',
      'ESTADO',
      'DDD1',
      'TELEFONE1',
      'DDD2',
      'TELEFONE2',
      'EMAIL',
      'CNPJ', 
      'RAZAO SOCIAL', #razao social
      'NOME FANTASIA', 
      'DT_ABERTURA', #Data de abertura
      'ANO_ABERTURA', #ANO de abertura
      'MES_ABERTURA', #MES de abertura
      'CNAE',
      'PORTE', #porte 00 - NAO INFORMADO 01 - MICRO EMPRESA 03 - EMPRESA DE PEQUENO PORTE 05 - DEMAIS
      'SITUACAO' #situacao 01 - NULA 02 - ATIVA 03 - SUSPENSA 04 - INAPTA 08 - BAIXADA
      ])

    situacao_dict ={
        1: 'Não Informada',
        '01': 'Não Informada',
        2: 'Ativa',
        '02': 'Ativa',
        3: 'Suspensa',
        '03': 'Suspensa',
        4: 'Inapta',
        '04': 'Inapta',
        8: 'Baixada',
        '08': 'Baixada'
    }
    for row in ws.rows:
        cpfs.append(str(row[0].value))

        if len(cpfs)<100:
            continue
        elif len(cpfs)==100:
            consulta =  tuple(cpfs)            
            cpfs.clear()
        print('consultado')
        print(consulta)
        # cnpj = re.sub('([.]|[-]|[/])', '', str(cnpj))
        # if(len(str(cnpj)) == 13):
        #     cnpj = '0' + str(cnpj)
        # elif(len(str(cnpj)) == 12):
        #     cnpj = '00' + str(cnpj)
        # elif(len(str(cnpj)) == 11):
        #     cnpj = '000' + str(cnpj)
        # elif(len(str(cnpj)) == 10):
        #     cnpj = '0000' + str(cnpj)
        # elif(len(str(cnpj)) == 9):
        #     cnpj = '00000' + str(cnpj)
        # elif(len(str(cnpj)) == 8):
        #     cnpj = '000000' + str(cnpj)
        #print('CNPJ =: ',str(cnpj))
        enriched_company = get_data_from_RFB(consulta)
        for j in range (len(enriched_company)):
            newFileWriter.writerow([str(enriched_company[j][0]), str(enriched_company[j][1]), str(enriched_company[j][2]), str(enriched_company[j][3]), str(enriched_company[j][4]), str(enriched_company[j][5]), str(enriched_company[j][6]), str(enriched_company[j][7]), str(enriched_company[j][8]), str(enriched_company[j][9]), str(enriched_company[j][10]), str(enriched_company[j][11]), str(enriched_company[j][12]), int(enriched_company[j][20]), str(enriched_company[j][13]), get_fantasy_name(str(enriched_company[j][13]),str(enriched_company[j][14])), str(enriched_company[j][15]), str(enriched_company[j][15])[0:4], str(enriched_company[j][15])[4:6], str(enriched_company[j][16]), get_porte(enriched_company[j][17], enriched_company[j][18]), situacao_dict[enriched_company[j][19]]])
        print('Escrito')
    #para o ultimo caso
    print('ultima consulta')
    consulta =  tuple(cpfs)            
    cpfd.clear()
    enriched_company = get_data_from_RFB(consulta)
    for j in range (len(enriched_company)):
        newFileWriter.writerow([str(enriched_company[j][0]), str(enriched_company[j][1]), str(enriched_company[j][2]), str(enriched_company[j][3]), str(enriched_company[j][4]), str(enriched_company[j][5]), str(enriched_company[j][6]), str(enriched_company[j][7]), str(enriched_company[j][8]), str(enriched_company[j][9]), str(enriched_company[j][10]), str(enriched_company[j][11]), str(enriched_company[j][12]), int(enriched_company[j][20]), str(enriched_company[j][13]), get_fantasy_name(str(enriched_company[j][13]),str(enriched_company[j][14])), str(enriched_company[j][15]), str(enriched_company[j][15])[0:4], str(enriched_company[j][15])[4:6], str(enriched_company[j][16]), get_porte(enriched_company[j][17], enriched_company[j][18]), situacao_dict[enriched_company[j][19]]])
    print('Escrito')
    
# fechando conexões e cursores
cursor_sql_server.close()
sql_server_conection.close()

# fechando planilha
wb.close()


Iniciando conexão com banco de dados
Conectado
Iniciando leitura da planilha de cpfs
consultado
('308163273', '1984665286', '2729914226', '2898613223', '3458804200', '3505411248', '7769905435', '9712887200', '11869030745', '21710112204', '29482769856', '32208898249', '41175077291', '44395639291', '51111128200', '57279020291', '69031967220', '70443777268', '71392286123', '74300300259', '74802453272', '76167798249', '78557879253', '79102964287', '82566836272', '85836566577', '90059891220', '97931128249', '98141333291', '208119779', '517494752', '852760493', '861807464', '1011362422', '1080002499', '1223121470', '1293252441', '1359895400', '1365831485', '1383021465', '1433640570', '1557922705', '1755338228', '1775490505', '1904422128', '1930218460', '1968186484', '2004019476', '2038276439', '2087026440', '2175888401', '2307575456', '2307575456', '2325553469', '2332755411', '2407087410', '2481792505', '2487215429', '2519469404', '2592717480', '2655081501', '2751942482', '2800146435', '2814